# 第21章 模块：宏伟蓝图

- 模块是最高级别的程序组织单元，它将程序代码和数据封装起来以便重用；模块往往对应于Python程序文件

## 为什么使用模块

- 代码重用
- 系统命名空间的划分：模块将变量名封装进了自包含的软件包，这一点对避免变量名的冲突很有帮助
- 实现共享服务和数据

## 导入和属性

- 就在import执行时，import会一次运行在目标文档中的语句从而建立其中的内容

## import如何工作

- 同C++中的`#include`相比，在Python中，导入并非只是把一个文件文本插入另一个文件而已，导入其实是运行时的运算，程序第一次导入指定文件时，会执行三个步骤：1.找到模块文件；2.编译成位码（需要时）；3.执行模块的代码来创建其所定义的对象
- 以上三个步骤只在程序执行时，模块第一次导入时才会进行，在这之后，导入相同模块时，会跳过这三个步骤，而只提取内存中已加载的模块对象；从技术上讲，Python把载入的模块存储到一个名为sys.modules的表中，并在一次导入操作的开始检查该表，如果模块不存在，将会启动一个三步骤的过程
- 如果你在模块已加载后还需要再次导入，就得通过调用reload强制处理这个问题

### 搜索

- 模块导入搜索路径由以下四个部分组成：1.程序的主目录，2.PYTHONPATH目录，3.标准链接库目录，4.任何.pth文件的内容（如果存在的话）；它们组成了sys.path
- 搜索路径的第一和第三元素是自动定义的，但是因为Python会从头到尾搜索这些组件组合的结果，第二和第四元素，就可以用于拓展路径，从而包含你自己的源代码目录
- **程序的主目录：**程序的顶层脚本文件所在目录（而非当前工作目录）；由于这个目录是先搜索的，其文件也将覆盖路径上的其它目录中具有同样名称的模块，如果你需要在自己的程序中使用库模块的话，小心不要以这种方式意外地隐藏库模块
- **PYTHONPATH目录：**可以把想导入的目录加到这个环境变量中，Python会从左至右搜索PYTHONPATH设置中罗列出的所有目录
- **标准库目录：**标准库模块安装在机器上的那些目录
- **.pth文件目录：**在后缀名为.pth的文本文件中一行一行地列出目录

- 如果你想看看模块搜索路径在机器上的实际配置，可以打印sys.path列表，导入时，Python会由左至右搜索这个列表中的每个目录
- 通过修改sys.path列表，可以修改导入搜索路径，当然这种修改只在脚本运行期间生效

### 编译（可选）

- 找到符合import语句的源代码文件后，Python会检查文件的时间戳，如果发现字节码文件比源码文件旧（例如，源码被修改过），就会重新生成字节码，否则不会；此外，如果Python在搜索路径中只发现了字节码，而没有源码，就会直接加载字节码（这意味着你可以把一个程序只作为字节码文件发布，而避免发送源码）；换句话说，如果有可能使程序的启动提速，就会跳过编译步骤
- 只有被导入的文件才会在机器上留下.pyc，顶层文件的字节码是在内部使用后就丢弃了，被导入文件的字节码则保存在文件中从而可以提高之后的导入速度

### 运行

- import操作的最后步骤是执行模块的字节码，文件中所有语句会依次执行，此步骤中任何对变量名的赋值运算，都会产生所得到的模块文件的属性
- 如果模块文件中任何顶层代码确实做了什么实际的工作，你就会在导入时看见其结果（例如，print语句）

## 模块文件选择

- 文件名的后缀（例如，.py）是刻意从import语句中省略的，这是一种对导入机制的延伸，让导入者完全忽略需要加载的文件类型之间的差异；例如，`import b`形式的import可能会加载：1.源代码文件b.py，2.字节码文件b.pyc，3.目录b，包导入，4.编译扩展模块（通常用C或C++编写）等等
- 如果在相同目录中找到b.py和b.so，Python会遵循一个标准的挑选顺序，不过这种顺序并不固定